<a href="https://colab.research.google.com/github/benjamint24/DataRoyaleFall2025/blob/main/exploration_w_gpu_IDnames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
!nvidia-smi  # GPU VRAM (e.g., ~16 GB on T4)


Wed Nov 12 22:40:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   33C    P0             55W /  400W |   60125MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [72]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
csv_path = "/content/drive/MyDrive/BIG DATA/battles.csv"
parquet_path = "/content/drive/MyDrive/BIG DATA/battles.parquet"


In [74]:
import cudf

gdf = cudf.read_csv(csv_path)
print(len(gdf), "rows")
gdf.head()


MemoryError: std::bad_alloc: out_of_memory: CUDA error (failed to allocate 403629824 bytes) at: /__w/rmm/rmm/cpp/include/rmm/mr/device/cuda_memory_resource.hpp:62: cudaErrorMemoryAllocation out of memory

In [ ]:
gdf.tail()

In [ ]:
gdf = gdf.drop(columns=['Unnamed: 0', 'battleTime', 'gameMode.id',
                 'winner.clan.tag', 'winner.clan.badgeId',
                 'loser.clan.tag', 'loser.clan.badgeId', 'tournamentTag'])


In [ ]:
#List of columns to convert
id_columns = [
    "winner.cards.list","winner.card1.id","winner.card2.id","winner.card3.id","winner.card4.id",
    "winner.card5.id","winner.card6.id","winner.card7.id","winner.card8.id",
    "loser.cards.list","loser.card1.id","loser.card2.id","loser.card3.id","loser.card4.id",
    "loser.card5.id","loser.card6.id","loser.card7.id","loser.card8.id"
]

#Convert only these columns to string
for col in id_columns:
    gdf[col] = gdf[col].astype(str)
id_to_name = {
    "archers": 26000001,
    "arrows": 28000001,
    "baby-dragon": 26000015,
    "balloon": 26000006,
    "bandit": 26000046,
    "barbarian-hut": 27000005,
    "barbarians": 26000008,
    "bats": 26000049,
    "battle-ram": 26000036,
    "bomb-tower": 27000004,
    "bomber": 26000013,
    "bowler": 26000034,
    "cannon": 27000000,
    "cannon-cart": 26000054,
    "clone": 28000013,
    "dark-prince": 26000027,
    "dart-goblin": 26000040,
    "electro-wizard": 26000042,
    "elite-barbarians": 26000043,
    "elixir-collector": 27000007,
    "executioner": 26000045,
    "fire-spirits": 26000031,
    "fireball": 28000000,
    "flying-machine": 26000057,
    "freeze": 28000005,
    "furnace": 27000010,
    "giant": 26000003,
    "giant-skeleton": 26000020,
    "goblin-barrel": 28000004,
    "goblin-gang": 26000041,
    "goblin-hut": 27000001,
    "goblins": 26000002,
    "golem": 26000009,
    "graveyard": 28000010,
    "guards": 26000025,
    "heal": 28000016,
    "hog-rider": 26000021,
    "hunter": 26000044,
    "ice-golem": 26000038,
    "ice-spirit": 26000030,
    "ice-wizard": 26000023,
    "inferno-dragon": 26000037,
    "inferno-tower": 27000003,
    "knight": 26000000,
    "lava-hound": 26000029,
    "lightning": 28000007,
    "lumberjack": 26000035,
    "mega-knight": 26000055,
    "mega-minion": 26000039,
    "miner": 26000032,
    "mini-pekka": 26000018,
    "minion-horde": 26000022,
    "minions": 26000005,
    "mirror": 28000006,
    "mortar": 27000002,
    "musketeer": 26000014,
    "night-witch": 26000048,
    "pekka": 26000004,
    "poison": 28000009,
    "prince": 26000016,
    "princess": 26000026,
    "rage": 28000002,
    "rocket": 28000003,
    "royal-ghost": 26000050,
    "royal-giant": 26000024,
    "skeleton-army": 26000012,
    "skeleton-barrel": 26000056,
    "skeletons": 26000010,
    "sparky": 26000033,
    "spear-goblins": 26000019,
    "tesla": 27000006,
    "the-log": 28000011,
    "three-musketeers": 26000028,
    "tombstone": 27000009,
    "tornado": 28000012,
    "valkyrie": 26000011,
    "witch": 26000007,
    "wizard": 26000017,
    "x-bow": 27000008,
    "zap": 28000008,
    "zappies": 26000052
}
# This line converts the original dictionary to have integer IDs as keys and card names as values
id_to_name = {v: k for k, v in id_to_name.items()}
# Convert the keys of the id_to_name dictionary to strings to match the column data type
id_to_name_for_replace = {str(k): v for k, v in id_to_name.items()}

for col in id_columns:
  if "cards.list" in col:
        # Handle list columns — convert each ID in the list
        gdf[col] = gdf[col].apply(
            lambda x: [
                id_to_name_for_replace.get(str(i), str(i))
                for i in eval(x) if x not in [None, "nan", "None"]
            ] if x.startswith("[") else x
        )
  else:
    # Use the dictionary with string keys for replacement of single-value ID columns
    gdf[col] = gdf[col].replace(id_to_name_for_replace)
gdf.head()
print(gdf[["loser.card1.id","winner.cards.list"]].head())

In [ ]:
gdf.info()

In [ ]:
null_summary = gdf.isnull().sum().to_pandas()  # move small result to CPU for display
null_summary = null_summary[null_summary > 0]
print(null_summary)


In [ ]:
# How many wins where loser.kingTowerHitPoints is null
gdf[gdf['loser.kingTowerHitPoints'].isna()]['winner.crowns'].value_counts()
#null means tower destroyed

In [ ]:
# Check winner's princess towers
gdf[gdf['winner.princessTowersHitPoints'].isna()]['winner.crowns'].value_counts()

# Check loser's princess towers
gdf[gdf['loser.princessTowersHitPoints'].isna()]['winner.crowns'].value_counts()

#winner.princessTowersHitPoints     19640
#loser.kingTowerHitPoints          274647
#loser.princessTowersHitPoints     510829

In [ ]:
gdf['winner.princessTowersHitPoints'] = gdf['winner.princessTowersHitPoints'].fillna(0)
gdf['loser.kingTowerHitPoints'] = gdf['loser.kingTowerHitPoints'].fillna(0)
gdf['loser.princessTowersHitPoints'] = gdf['loser.princessTowersHitPoints'].fillna(0)

In [ ]:
gdf.duplicated().sum()

In [ ]:
gdf = gdf.drop_duplicates()

In [ ]:
gdf['loser.cards.list']

In [ ]:
import cudf
from cudf.core.dtypes import ListDtype

WIN_COL = "winner.cards.list"
LOS_COL = "loser.cards.list"

def ensure_listcol(s: cudf.Series) -> cudf.Series:
    """
    Convert string-encoded lists like "[26000000, 26000001, ...]" to a cuDF ListColumn.
    Assumes items are numeric IDs (no commas inside a single id).
    """
    if isinstance(s.dtype, ListDtype):
        return s
    s = s.astype("string").fillna("")
    s = (
        s.str.strip()
         .str.strip("[]")
         .str.replace('"', "", regex=False)
         .str.replace("'", "", regex=False)
    )
    s = s.replace("", None)
    return s.str.split(",")

# 1) Build list columns on GPU (no CPU round-trip)
w_list = ensure_listcol(gdf[WIN_COL])
l_list = ensure_listcol(gdf[LOS_COL])

# 2) Canonicalize decks by sorting each list ON GPU (same as sorted(lst))
w_sorted = w_list.list.sort_values()
l_sorted = l_list.list.sort_values()

# 3) Extract fixed 8 positions to columns (fast numeric groupby)
def to_deck_matrix(lst: cudf.Series) -> cudf.DataFrame:
    out = cudf.DataFrame({f"c{i}": lst.list.get(i) for i in range(8)})
    # Cast to int if possible
    for c in out.columns:
        out[c] = out[c].astype("int64", errors="ignore")
    return out

w_decks = to_deck_matrix(w_sorted)
l_decks = to_deck_matrix(l_sorted)

# Optionally require full 8-card decks
full_w = ~w_decks.isnull().any(axis=1)
full_l = ~l_decks.isnull().any(axis=1)
w_decks = w_decks[full_w]
l_decks = l_decks[full_l]

key_cols = [f"c{i}" for i in range(8)]

# 4) Count wins/losses per canonical deck (GPU groupby on 8 int columns)
wins   = w_decks.groupby(key_cols).size().rename("wins")
losses = l_decks.groupby(key_cols).size().rename("losses")

# 5) Join & compute totals / win_rate (GPU)
deck_stats = wins.to_frame().join(losses.to_frame(), how="outer").fillna(0)
deck_stats["wins"]   = deck_stats["wins"].astype("int64")
deck_stats["losses"] = deck_stats["losses"].astype("int64")
deck_stats["total"]  = deck_stats["wins"] + deck_stats["losses"]
deck_stats["win_rate"] = (deck_stats["wins"] / deck_stats["total"]).fillna(0.0)

# 6) Filter/sort (GPU)
min_games = 20
deck_stats = deck_stats[deck_stats["total"] >= min_games] \
                 .sort_values(["total", "win_rate"], ascending=[False, False])

print(deck_stats.head())


In [ ]:
#what classifies as a good deck,
top_decks = deck_stats.sort_values('win_rate', ascending=False).head(10)
popular_decks = deck_stats.sort_values('total', ascending=False).head(10)
underperforming = deck_stats[(deck_stats['total'] > 50) & (deck_stats['win_rate'] < 0.45)]
overperforming = deck_stats[(deck_stats['total'] > 50) & (deck_stats['win_rate'] > 0.6)]


In [ ]:
overperforming

In [ ]:
from cudf.core.dtypes import ListDtype

WIN_COL = "winner.cards.list"
LOS_COL = "loser.cards.list"
TOP_K   = 1000  # adjust as desired

def ensure_listcol(s: cudf.Series) -> cudf.Series:
    """Convert string-encoded lists like 'a,b,c' or "['a','b']" to ListColumn."""
    if isinstance(s.dtype, ListDtype):
        return s
    s = s.astype("string").fillna("")
    s = (
        s.str.strip()
         .str.strip("[]")
         .str.replace('"', "", regex=False)
         .str.replace("'", "", regex=False)
    )
    s = s.replace("", None)
    return s.str.split(",")

# 1) Convert to list columns (GPU)
w_list = ensure_listcol(gdf[WIN_COL])
l_list = ensure_listcol(gdf[LOS_COL])

# 2) Explode (older cuDF uses .explode())
w_ex = w_list.explode().astype("string").str.strip()
l_ex = l_list.explode().astype("string").str.strip()

# 3) Drop nulls/empties
w_ex = w_ex.dropna()
l_ex = l_ex.dropna()
w_ex = w_ex[w_ex != ""]
l_ex = l_ex[l_ex != ""]

# 4) Count wins/losses on GPU
wins = w_ex.value_counts(); wins.name = "w"
losses = l_ex.value_counts(); losses.name = "l"

# 5) Join + compute impact (GPU)
all_counts = wins.to_frame().join(losses.to_frame(), how="outer").fillna(0)
den = all_counts["w"] + all_counts["l"]
all_counts["impact"] = all_counts["w"] / den

# 6) Sort/top-K and (optionally) collect to CPU
card_impact_gdf = all_counts[["impact"]].sort_values("impact", ascending=False).head(TOP_K)



In [ ]:
card_impact_gdf

In [ ]:
# Combine winner + loser card lists
all_cards = cudf.concat([w_list.explode(), l_list.explode()])

# Drop nulls/empties and clean
all_cards = all_cards.dropna().astype("string").str.strip()
all_cards = all_cards[all_cards != ""]

# Count frequencies on GPU
freq_df = all_cards.value_counts().rename("count").to_frame()

print("Top 10 most used cards:")
print(freq_df.head(10))


In [ ]:
import cudf
from cudf.core.dtypes import ListDtype

WIN_COL = "winner.cards.list"
LOS_COL = "loser.cards.list"

def ensure_listcol(s: cudf.Series) -> cudf.Series:
    """
    Convert string-encoded lists like "[26000000, 26000001, ...]" to a cuDF ListColumn.
    Assumes items are numeric IDs (no internal commas within an item).
    """
    if isinstance(s.dtype, ListDtype):
        return s
    s = s.astype("string").fillna("")
    s = (
        s.str.strip()
         .str.strip("[]")
         .str.replace('"', "", regex=False)
         .str.replace("'", "", regex=False)
    )
    s = s.replace("", None)
    return s.str.split(",")

# 1) Build list columns on GPU
w_list = ensure_listcol(gdf[WIN_COL])
l_list = ensure_listcol(gdf[LOS_COL])

# 2) Canonicalize: sort each deck list on GPU
w_sorted = w_list.list.sort_values()
l_sorted = l_list.list.sort_values()

# 3) Extract the 8 cards into fixed columns (Clash decks are size 8)
#    If some rows don’t have 8, missing positions will be null (handled below).
def to_deck_matrix(lst: cudf.Series) -> cudf.DataFrame:
    out = cudf.DataFrame({f"c{i}": lst.list.get(i) for i in range(8)})
    # cast to int if possible
    for c in out.columns:
        out[c] = out[c].astype("int64", errors="ignore")
    return out

w_decks = to_deck_matrix(w_sorted)
l_decks = to_deck_matrix(l_sorted)

# 4) Drop any rows that don't have a full deck (optional; or keep and they'll group separately)
full_row_mask_w = ~w_decks.isnull().any(axis=1)
full_row_mask_l = ~l_decks.isnull().any(axis=1)
w_decks_full = w_decks[full_row_mask_w]
l_decks_full = l_decks[full_row_mask_l]

key_cols = [f"c{i}" for i in range(8)]

# 5) Count wins/losses per canonical deck (GPU)
wins = w_decks_full.groupby(key_cols).size().rename("wins")
losses = l_decks_full.groupby(key_cols).size().rename("losses")

# 6) Join and compute totals/rates (GPU)
deck_stats = wins.to_frame().join(losses.to_frame(), how="outer").fillna(0)
deck_stats["wins"]   = deck_stats["wins"].astype("int64")
deck_stats["losses"] = deck_stats["losses"].astype("int64")
deck_stats["total"]  = deck_stats["wins"] + deck_stats["losses"]
deck_stats["win_rate"] = (deck_stats["wins"] / deck_stats["total"]).fillna(0.0)

# 7) Filter/sort as before (GPU)
min_games = 20
deck_stats = deck_stats[deck_stats["total"] >= min_games] \
                 .sort_values(["total", "win_rate"], ascending=[False, False])

print(deck_stats.head())
